<a href="https://colab.research.google.com/github/sir-sauc3/cap-comp215/blob/main/Sos%26RobertoProject2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sos' and Roberto's Project 2
##A Simple Grazing Model

####Objectives
* to develop a Cellular Automata model of grazing
* to experiment with intervention policies, like culls to examine their effect

####Pre-req
* Review Ch. 8 Diffusion and Ch. 11.5: Sugarscape (from textbook On Complexity)
* Complete Labs 7-8 – this provides the framework used for this project

In [18]:
import numpy as np
from dataclasses import dataclass
from pprint import pprint
import random

import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation

In [2]:
class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number

      See https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/
          for a discussion of the pros and cons of various animation techniques in jupyter notebooks
    """

    def __init__(self, model, frames=50, steps_per_frame=1, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.steps_per_frame = steps_per_frame
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            for _ in range(self.steps_per_frame):
                self.model.step()
        self.model.draw(self.ax, step=step*self.steps_per_frame)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()          # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)     # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()               # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass

In [24]:
class GrazingSimulation:

  kernel = np.array([ [0,0,0],
                      [0,0,0],
                      [0,0,0]
                   ])

  def __init__(self, size, coverage, deer_amt, deer_weight):
    self.grassland = np.random.choice((1,0), (size, size), p=(coverage, 1-coverage))
  
    field = np.random.choice(('G','E'), (50,50), p=(0.5, 0.5))
    movementRate : int
    weightLossRate : int
    M : int
    N = random.randint(20,40)
    defaultWeight = 20

  # def draw(self, axes=None, step=''):
  #   """Draws the cells on given axes."""
  #   axes = axes or plt.gca()  # use current axes if none are provided.
  #   plot_2D_array(self.state, axes, cmap='YlOrRd', vmax=5,  title=f'Sandpile {step}')

  def plot_grass_deer(grass, deer):
    fig, ax = plt.subplots()
    ax.imshow(grass, cmap='Greens')
    deer_xs, deer_ys = np.where(deer > 0)
    ax.scatter(deer_ys, deer_xs, marker='o', color='brown')
    plt.show()

In [ ]:
grassland_size = 10
grass_coverage = 0.5 #between 0 <= grass_coverage <= 1
no_deer = 10
deer_weight = 100

GrazingSimulation(grassland_size, grass_coverage, no_deer, deer_weight)

In [4]:
# test = Grazing2D(1,1,1)
# # print(test.field)

# model = Animation2D(test)
# model.animate()

In [5]:
#STUFF I NEED TO DO

# 1. Create a grassland
# 2. Find a way to introduce deer onto the grassaland (each with a weight index of W)
# 3. Find a way to make the deer move via a random walk
# 4. When deer graze, the weight index increases to M (i guess i can choose the rate at which it increases)
# 5. When its not grazing (looking for food), W decreases (i guess i chose this rate too)
# 6. Deer dies when W gets to 0


# HOW IM GOING TO DO IT

# 1. create an np array and randomly fill it from 0-3. 0 for no grass, 3 for most grass. using np.random.choice()

In [6]:
# grassland = np.random.choice([0,1,2,3], (15,15), p=[0.5, 0.3, 0.1, 0.1])

# print(grassland)

[[1 1 1 0 0 2 1 0 0 1 1 0 0 2 0]
 [3 1 0 1 1 0 0 1 0 0 1 3 1 1 2]
 [1 0 1 1 1 1 0 0 0 0 1 1 0 1 3]
 [2 1 0 2 0 2 0 1 1 1 0 1 1 0 3]
 [0 2 0 2 0 1 0 3 0 2 1 1 0 3 0]
 [0 1 0 1 1 1 0 3 3 0 2 0 1 2 0]
 [0 3 0 0 0 1 3 3 0 0 1 0 0 3 0]
 [1 2 0 0 0 2 1 1 0 0 1 2 0 0 0]
 [0 3 1 0 0 0 0 0 3 0 0 0 0 0 3]
 [2 0 0 2 1 0 0 2 0 3 3 1 0 0 0]
 [0 0 1 2 0 2 0 0 1 0 1 1 0 0 0]
 [0 0 3 2 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 1 0 0 3 1 1 0 1 0 0 0 1 2]
 [3 0 0 1 0 0 3 1 2 0 1 0 1 0 0]
 [0 1 0 0 0 0 1 1 1 0 0 2 0 0 0]]
